# 融合core与qmof
本note旨在融合core与qmof数据集，进行数据的初步处理

In [1]:
import pandas as pd

def dropun(X):
    for x in X.columns:
        if x[:7]=='Unnamed':
              X=X.drop(columns=[x])
    return X

## 读入core数据集

In [2]:
core = dropun(pd.read_csv("CoRE_database/CoRE_MOFs.csv"))
print(core.shape)

# core.columns=['name']
core_name = list(core['name'])

(10140, 6)


In [3]:
cnt = 0
for mof in core_name:
    if mof[0]<'A' or mof[0]>'Z':
        cnt +=1
print(cnt, cnt/len(core_name))


415 0.040927021696252466


## 读入qmof数据集

In [4]:
qmof = dropun(pd.read_csv("qmof_database/qmof.csv"))

print(qmof.shape)

(18321, 21)


In [5]:
qmof.isnull().describe()


,name,info.source,info.formula,info.natoms,info.pld,info.lcd,info.density,inputs.pbe.theory,inputs.pbe.pseudopotentials,inputs.pbe.encut,...,inputs.pbe.gamma,inputs.pbe.spin,outputs.pbe.energy_total,outputs.pbe.energy_vdw,outputs.pbe.energy_elec,outputs.pbe.bandgap,outputs.pbe.directgap,outputs.pbe.cbm,outputs.pbe.vbm,outputs.pbe.net_magmom
count,18321,18321,18321,18321,18321,18321,18321,18321,18321,18321,...,18321,18321,18321,18321,18321,18321,18321,18321,18321,18321
unique,1,1,1,1,2,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,18321,18321,18321,18321,18319,18319,18321,18321,18321,18321,...,18321,18321,18321,18321,18321,18321,18321,18321,18321,18321


In [6]:
for i in range(qmof.shape[0]):
    if qmof.loc[i, 'name'].endswith('_FSR'):
        qmof.loc[i, 'name'] = qmof.loc[i, 'name'][:-4]

In [7]:
'ABAVIJ' in qmof['name']

False

In [8]:
core_qmof = pd.merge(core, qmof, how='inner', left_on='name', right_on='name')

In [9]:
core_qmof.describe()

,Deposition Number(s),info.natoms,info.pld,info.lcd,info.density,inputs.pbe.encut,outputs.pbe.energy_total,outputs.pbe.energy_vdw,outputs.pbe.energy_elec,outputs.pbe.bandgap,outputs.pbe.cbm,outputs.pbe.vbm,outputs.pbe.net_magmom
count,1.231000e+03,1294.000000,1294.000000,1294.000000,1294.000000,1294.0,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000
mean,8.695718e+05,160.122102,3.008050,4.371541,1.662960,520.0,-1076.720700,-11.807468,-1064.913231,2.107698,1.382659,-0.725039,3.042087
std,3.027292e+05,92.481359,1.833792,1.852618,0.449311,0.0,617.950760,6.705643,611.423905,1.225390,1.135637,1.008547,8.542077
min,1.013380e+05,24.000000,0.583490,1.231010,0.475276,520.0,-3449.382160,-37.828250,-3413.768901,0.003537,-4.315972,-4.722541,0.000000
25%,7.333245e+05,94.000000,1.672273,3.171125,1.339884,520.0,-1390.554230,-14.796610,-1374.470651,0.931663,0.621968,-1.381533,0.000000
50%,8.751770e+05,136.000000,2.698560,4.129870,1.581588,520.0,-932.175776,-10.318445,-921.094991,2.275854,1.420754,-0.741766,0.000000
75%,9.980655e+05,208.000000,3.743205,5.039013,1.927021,520.0,-629.457388,-7.112253,-621.970803,3.037810,2.152122,-0.098097,0.000000
max,1.542564e+06,500.000000,16.924860,16.989590,4.051389,520.0,-140.843654,-1.669500,-138.669684,5.306719,5.139415,2.687465,100.000000


In [10]:
core_qmof.to_csv('core_qmof.csv')

以上融合的结果为1294个，保存在core_qmof中

# 导出core_qmof链接

下载后数据集pdf名称对照表被存在core_qmof_pdf中，doi的最后一栏就是split（/）之后取str[4]

由于core_qmof_pdf是core_qmof的简化版，导出core_qmof_link之后需要在此与core_qmof_pdf联合

注意：
1. **doi及文件名要统一大写转小写**
2. **doi及文件名要统一将%转成正常字符！**

In [11]:
def get_filename(name):
    return name.lower().replace('%28', '(').replace('%29', ')').replace('%3b', ';').replace('%3a', ':')

In [12]:
import os

def rename (path):
    filenames = os.listdir(path)
    for filename in filenames:
        lowerfilename = get_filename(filename)
        file = os.path.join(path, filename)
        new_file = os.path.join(path, lowerfilename)
        if os.path.isdir(file):
            rename(file)
        else:
            os.rename(file, new_file)
    
for directory in os.listdir('../convert/'):
    if os.path.isdir('../convert/' + directory) and directory.startswith('core_qmof'):
        print(directory)
        rename('../convert/' + directory)

core_qmof_word
core_qmof_html_dest
core_qmof_pdf
core_qmof_html
core_qmof_txt_word
core_qmof_txt_original
core_qmof_txt_pdfminer
core_qmof_txt


In [13]:
core_qmof_pdf = dropun(pd.read_csv('core_qmof_pdf.csv'))  # 丢失了数据

![image-20210802190517360](https://gitee.com/sun__ye/gallery/raw/master/img2/20210802190542.png)

![image-20210802190353879](https://gitee.com/sun__ye/gallery/raw/master/img2/20210802190550.png)

![image-20210802190409676](https://gitee.com/sun__ye/gallery/raw/master/img2/20210802190552.png)

df自动将 `%28`转为`(`，自动将`%29`转为`)`，而文件名却不会自动转换。因此再通过dataframe寻找对应的文件时找不到。然而后续文件名中只要带有%的文件，zdir都无法访问，这里必须还是按照dataframe的方式。将doi和另存为的文件名都转一下

In [14]:
untrace = pd.DataFrame(columns=['name', 'doi_link'])
os.system('rm -rf ../convert/untraced_pdf')

for i in range(core_qmof_pdf.shape[0]):
#     print(core_qmof_pdf.loc[i,'doi_link'])
    if core_qmof_pdf.loc[i,'doi_link'] == 'not_exist':
        core_qmof_pdf.loc[i,'doi'] = 'not_exist'
        continue
        
    doi = core_qmof_pdf.loc[i,'doi_link'].split('/')[4]

    doi = get_filename(doi)
    core_qmof_pdf.loc[i,'doi'] = doi
    if os.path.exists('../convert/core_qmof_pdf/' + doi + '.pdf'):
        core_qmof_pdf.loc[i,'pdf_link'] = 'http://d.sunie.top:9009/material/convert/core_qmof_pdf/' + doi + '.pdf'
    else:
        core_qmof_pdf.loc[i,'pdf_link'] = 'not_exist'
        untrace.loc[untrace.shape[0]]={'name': core_qmof_pdf.loc[i, 'name'], 'doi_link': core_qmof_pdf.loc[i,'doi_link']}
        print(doi)

    if os.path.exists('../convert/core_qmof_txt/' + doi + '.txt'):
        # 这里不需要后缀名！！
        core_qmof_pdf.loc[i, 'brat_link'] = 'http://d.sunie.top:9020/index.xhtml#/material/convert/core_qmof_txt/' + doi
    else:
        core_qmof_pdf.loc[i, 'brat_link'] = 'not_exist'
        
    if os.path.exists('../convert/core_qmof_txt_pdfminer/' + doi + '.txt'):
        core_qmof_pdf.loc[i, 'pdfminer_link'] = 'http://d.sunie.top:9009/material/convert/core_qmof_txt_pdfminer/' + doi + '.txt'
    else:
        core_qmof_pdf.loc[i, 'pdfminer_link'] = 'not_exist'
    
    if os.path.exists('../convert/core_qmof_html/' + doi + '.html'):
        core_qmof_pdf.loc[i,'html_link'] = 'http://d.sunie.top:9009/material/convert/core_qmof_html/' + doi + '.html'
    else:
        core_qmof_pdf.loc[i,'html_link'] = 'not_exist'
        
        # 记录未转换的文件
        os.makedirs('../convert/untraced_pdf', exist_ok=True)
        os.system(f'cp ../convert/core_qmof_pdf/{doi}.pdf ../convert/untraced_pdf/{doi}.pdf')

(sici)1521-3773(19990917)38:18%3c2730::aid-anie2730%3e3.3.co;2-0
1521-3749(200102)627:2%3c131::aid-zaac131%3e3.3.co;2-r
(sici)1521-3773(19990115)38:1


In [15]:
untrace.to_csv('untrace.csv')

In [16]:
core_qmof_pdf = core_qmof_pdf.drop(columns=['doi_link'])

In [17]:
core_qmof_link = pd.merge(core_qmof_pdf, core_qmof, how='inner', left_on='name', right_on='name')

In [18]:
core_qmof_link.to_csv('core_qmof_link.csv')

关于core_qmof_pdf有效数据的长度，必须排除`nan`在外！

In [19]:
len(set(core_qmof_pdf['doi']))

891

# 筛选出论文材料1:1的数据

1. 寻找只被指向过一次的论文
2. 根据这些论文寻找对应的材料，并添加

In [20]:
paper = {}

for i in range(len(core_qmof_link)):
    name = core_qmof_link.loc[i, 'name']
    doi_link = core_qmof_link.loc[i, 'doi_link']
    if doi_link!="not_exist":
        if doi_link not in paper.keys():
            paper[doi_link] = 0
        paper[doi_link] += 1

paper = [k for k,v in paper.items() if v==1 ]

In [21]:
drop_row = []
for i in range(len(core_qmof_link)):
    if core_qmof_link.loc[i, 'doi_link'] not in paper:
        drop_row.append(i)
        
core_qmof704 = core_qmof_link.drop(drop_row)

In [22]:
core_qmof704 = core_qmof704.reset_index().drop(columns=['index'])

In [23]:
core_qmof704.to_csv('core_qmof704.csv')

In [24]:
len(set(core_qmof704['pdf_link']))   # 实际pdf数量

702

# 删除转换较差的html

In [1]:
bad_doi = """ic802445q
chem.201002088
acs.cgd.5b00681
c0dt00931h
c0dt00999g
acs.inorgchem.5b02340
c6cc03285k
chem.200305098
acs.cgd.6b00890
adfm.200800058
ic00136a027
chem.201502033
ajchem.2014.16986
jssc.2001.9118
b200213b
c1dt10901d
s0108768101020717
ejic.201200686
j.solidstatesciences.2004.03.007
b809557d
chem.200801948
"""

bad_doi = bad_doi.split()

In [8]:
dirname = '../convert/core_qmof_html'

for f in os.listdir(dirname):
    if f.endswith('.html'):
        if f[:-5].endswith('(2)'):
            print(f)
            os.remove(f'{dirname}/{f}')

In [ ]:
import os

for doi in bad_doi.split():
    path = f'{dirname}/{doi}.html'
    os.remove(path)